In [95]:
# Rev 2 - added send mail to perx opt modile

In [96]:
import pandas as pd
import datetime as dt
import numpy as np
import psycopg2
import io
import os
import time
import warnings
import math
import csv
from IPython.display import clear_output

from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

import openpyxl as xl
from openpyxl.styles import *
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import rows_from_range

# from pretty_html_table import build_table
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from os.path import basename
import smtplib
import msal
import requests

In [97]:
# show all column
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format',  '{:,}'.format)

In [98]:
#### Step 1 add file db_config.py ลงไปใน หน้าหลักของ notebook ###
#### add config ###
import sys
#### ถอย folder มาหา file py ####
sys.path.append('/home/jovyan/shared')

In [99]:
import db_config

In [100]:
os.chdir('/home/jovyan/48_lead_generation/')

## Variable Config

In [101]:
## เรียก config ต่างๆ
dws_config = db_config.dws_config_o()
api_config = db_config.msgraph_config()

## เรียก token config
CLIENT_ID = api_config.client_id
CLIENT_SECRET = api_config.client_secret
TENANT_ID = api_config.tenant_id
SITE_ID = api_config.site_id
# Authority URL and scope for Microsoft Graph.
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["https://graph.microsoft.com/.default"]

## directory of share drive
current_year = str(datetime.now().year)
team_directory = r'General/lead_data'
sharepoint_directory = r'https://siampiwat1.sharepoint.com/sites/LeadSharingHub/Shared%20Documents/General/lead_data'

## file path
file_path = r'/home/jovyan/48_lead_generation/temp_files'

# Email details set reciept and sender here
sender_email = "DPM_automated_email@velocitytech.biz"
password = "password"
# receiver_email = "campaign_ops_cx@velocitytech.biz"
receiver_email = "nutthapol.w@velocitytech.biz"
### กรณีคนนอก อย่าลืมเปลี่ยน reciever email แต่พ่วง campaign ops ไว้ด้วย
# reciever_email = 'campaign_ops_cx@velocitytech.biz,others_person@iconsiam.com'
cc_email = "nutthapol.w@velocitytech.biz"

### For Perx Opt Email
# perx_opt_email = "pornthip.s@supremo.co.th"
perx_opt_email = "nutthapol.w@velocitytech.biz"
rcpt = cc_email.split(",") + receiver_email.split(",")

# lead choice and provider
lead_choice = ['1','2','3','4']
corp_choice = ['1','2','3','4']

## ประกาศ Function

In [102]:
## function สำหรับบันทึกว่าใครส่ง lead
# def sql_provider():
#     sql_provide = input(r'''Paste your SQL Script, Please make sure that no error in your SQL :\n''')
#     return sql_provide

In [103]:
### สำหรับ Query DWS Postgressql
def query_postgres(sql_text):
    #PostGresSQL Zone
    pg_conn = psycopg2.connect(
    host = dws_config.host['3'][0],
    database = dws_config.db,
    user = dws_config.usr,
    password = dws_config.pw,
    port = dws_config.port)
    cur = pg_conn.cursor()
    return pd.read_sql_query(sql_text , pg_conn)

In [104]:
### function ใส่ชื่อ subject
def lead_subject_provide():
    subject_name = input("Please input subject name เอาจากหัว Ticket: ")
    return 'RE : ' + subject_name

In [105]:
### function เลือก lead
def choose_type_of_lead():
    global lead_choice
    lead_selection = '0'
    while lead_selection not in lead_choice:
       lead_selection = input("Please select type of lead generate :\n1 = sms only\n2 = app push only\n3 = social ads only\n4 = sms and app push\n") 

    if lead_selection == '1':
        lead_text = "You selected sms only"
        # print(lead_text)
    elif lead_selection == '2':
        lead_text = "You selected app push only"
        # print(lead_text)        
    elif lead_selection == '3':
        lead_text = "You selected social ads only"
        # print(lead_text)             
    elif lead_selection == '4':
        lead_text = "You selected sms and app push"
        # print(lead_text)           
        
    return lead_selection, lead_text

In [106]:
## function สำหรับบันทึกว่าใครส่ง lead
def lead_provider():
    lead_provide = input("Who generate this lead? : ")
    return lead_provide

In [107]:
## function อ่าน file ใน folder
def absolute_file_paths(directory):
    path = os.path.abspath(directory)
    return [file.path for file in os.scandir(path) if file.is_file()]

In [108]:
## function ลบ file
def delete_temp_files():
    global file_path
    list_files = absolute_file_paths(file_path)
    for file in list_files:
        os.remove(file)
        print(f"Deleted {file} completed")

In [109]:
## function ตั้งชื่อ file return ออกมาเป็น list
def naming_lead():
    global lead_selection
    global lead_text
    print(lead_text)
    file_name = input("Please naming your lead file : ")
    if lead_selection == '1':
        sms_name = file_name + "_sms.xlsx"
        print(f"Your file name is : {sms_name}")
        return [sms_name], file_name
    elif lead_selection == '2':
        app_push_name = file_name + "_app_push.xlsx"
        print(f"Your file name is : {app_push_name}")
        return [app_push_name], file_name
    elif lead_selection == '3':
        social_ads_name = file_name + "_social_ads.xlsx"
        print(f"Your file name is : {social_ads_name}")
        return [social_ads_name], file_name
    elif lead_selection == '4':
        sms_name = file_name + "_sms.xlsx"
        app_push_name = file_name + "_app_push.xlsx"
        print(f"Your file name is : {sms_name} and {app_push_name}")
        return [sms_name, app_push_name], file_name

In [110]:
### write excel ใน temp folder
def write_excel():
    global file_path
    global lead_selection
    global initial_df
    global file_names_list
    file_names_list = [os.path.join(file_path,file) for file in file_names_list]
    ##### เลือก write base จาก lead_selection #####
    if lead_selection == '1':
        temp_df1 = initial_df[~initial_df['profileid'].isna()]['profileid']
        temp_df1.to_excel(file_names_list[0], index=False)
    elif lead_selection == '2':
        temp_df1 = initial_df[~initial_df['account_id_ctm'].isna()]['account_id_ctm']
        temp_df1.to_excel(file_names_list[0], index=False)
    elif lead_selection == '3':
        temp_df1 = initial_df[~initial_df['email'].isna()]['email'] ##social ads
        temp_df1.to_excel(file_names_list[0], index=False)
    elif lead_selection == '4':
        temp_df1 = initial_df[~initial_df['profileid'].isna()]['profileid']
        temp_df1.to_excel(file_names_list[0], index=False)
        temp_df2 = initial_df[~initial_df['account_id_ctm'].isna()]['account_id_ctm']
        temp_df2.to_excel(file_names_list[1], index=False)

In [111]:
### create ที่วางไฟล์
def choose_corp():
    global corp_choice
    corp_selection = '0'
    while corp_selection not in corp_choice:
       corp_selection = input("Please select corporate :\n1 = SPWG\n2 = PRHG\n3 = ICON\n4 = SPS\n") 
    
    if corp_selection == '1':
        corp_text = "You selected SPWG"
        corp = "SPWG"
        print(corp_text)
    elif corp_selection == '2':
        corp_text = "You selected PRHG"
        corp = "PRHG"
        print(corp_text)
    elif corp_selection == '3':
        corp_text = "You selected ICON" 
        corp = "ICON"
        print(corp_text)
    elif corp_selection == '4':
        corp_text = "You selected SPS"
        corp = "SPS"
        print(corp_text)
        
    return corp_selection, corp

In [112]:
### Module for upload file
def upload_file():
    global current_year
    global team_directory
    global corp_directory
    global file_names_list
    
    file_name_for_email = []

    for file in file_names_list:
        if 'sms' in file:
            DESTINATION_FILENAME = os.path.join(team_directory, current_year, corp_directory, 'sms_email', file.split('/')[-1])
        elif 'app_push' in file:
            DESTINATION_FILENAME = os.path.join(team_directory, current_year, corp_directory, 'app', file.split('/')[-1])   
        elif 'social_ads' in file:
            DESTINATION_FILENAME = os.path.join(team_directory, current_year, corp_directory, 'social_ads', file.split('/')[-1])   
        
        # create list for email
        file_name_for_email.append(file.split('/')[-1])
        
        # Create a confidential client application
        app = msal.ConfidentialClientApplication(
            CLIENT_ID,
            authority=AUTHORITY,
            client_credential=CLIENT_SECRET,
        )

        # Acquire an access token using client credentials flow
        result = app.acquire_token_for_client(scopes=SCOPE)

        if "access_token" in result:
            access_token = result["access_token"]
            headers = {
                "Authorization": f"Bearer {access_token}",
                "Content-Type": "application/octet-stream"
            }

            # Construct the endpoint URL for uploading the file.
            # This uploads the file to the root of the site's drive. Adjust the path if needed.
            endpoint = f"https://graph.microsoft.com/v1.0/sites/{SITE_ID}/drive/root:/{DESTINATION_FILENAME}:/content"

            # Read the file content in binary mode.
            with open(file, 'rb') as f:
                file_content = f.read()

            # Use PUT request to upload the file.
            response = requests.put(endpoint, headers=headers, data=file_content)

            if response.ok:
                print(f"{file.split('/')[-1]} uploaded successfully!")
                # print(response.json())
                success_parameter = 1
            else:
                print("Error uploading file:", response.status_code, response.text)
                success_parameter = 0
        else:
            print("Error obtaining access token:", result.get("error"), result.get("error_description"))
            success_parameter = 0
    return success_parameter, file_name_for_email

In [113]:
### module send email
def send_email():
    global success_params
    global subject
    global lead_name
    global lead_provide
    global file_name_for_email
    global sharepoint_directory
    global corp_directory
    
    sms_share_dir = sharepoint_directory + '/' + current_year + '/' + corp_directory + '/' + 'sms_email'
    app_share_dir = sharepoint_directory + '/' + current_year + '/' + corp_directory + '/' + 'app'
    social_share_dir = sharepoint_directory + '/' + current_year + '/' + corp_directory + '/' + 'social_ads'
    
    if success_params == 1:
        subject_success = f"{subject} - Lead Generate Completed!!"
        
        ## define body
        body_success = f"""
        <html>
          <body>
            <p style = "font-family: AngsanaUPC;
                       font-size: 24px">
             Dear All,<br>
             Lead <b> {lead_name} </b> Generate Completed.<br>
             Total files - <b>{file_name_for_email}</b><br>
             Lead Provider is : <b>{lead_provide}</b>.<br>
             any questions please feel free to reach <b>{lead_provide}</b>.
             <br><br>
             <b>Please get lead in MS Team share drive - {corp_directory}.</b><br>
             <a href={sms_share_dir}>SMS and Email Link</a><br>
             <a href={app_share_dir}>App Push Link</a><br>
             <a href={social_share_dir}>Social Ads Link</a><br>
             
             Thank you & Best regards<br>
            </p>
          </body>
          </font>
        </html>
        """
        
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Cc'] = cc_email
        
        msg['Subject'] = subject_success
        text = MIMEText(body_success, 'html')
        msg.attach(text)
        
        # Sending the email
        try:
            smtp = smtplib.SMTP('smtprelay.siampiwat.com', 25)
            smtp.sendmail(sender_email, rcpt, msg.as_string())
            smtp.quit()
            print("Successfully sent email")
        except smtplib.SMTPException:
            print("Error: unable to send email")

In [114]:
def opt_lead_check():
    is_opt = input("Do you to mail mobile number separately to perx operation team? 1 = Yes, 0 = No  : ")
    return is_opt

In [115]:
def write_opt_excel():
    global file_path
    global is_opt
    
    file_name = file_names_list[0]
    file_name = file_name.split('.')[0] + '_mobile_number.' + file_name.split('.')[1]
    
    if is_opt == '1':
        print('You have selected to send file to PERX operation Team')
        temp_df3 = initial_df[~initial_df['mobile_number'].isna()]['mobile_number']
        temp_df3.to_excel(file_name, index=False)
    else:
        pass

    return os.path.join(file_path,file_name)

In [116]:
def send_email_opt():
    global success_params
    global lead_name
    global lead_provide
    global opt_file_name
    global is_opt
    if success_params == 1 and is_opt == '1':
        subject_success = f"Lead {lead_name} for Opt"
        
        ## define body
        body_success = f"""
        <html>
          <body>
            <p style = "font-family: AngsanaUPC;
                       font-size: 24px">
             Dear All,<br>
             Lead <b>{lead_name}</b> Generate Completed.<br>
             Please find mobile number in attached.<br>
             Lead Provider is : <b>{lead_provide}</b>.<br><br>
             any questions please feel free to reach {lead_provide}.
             <br><br>
             Thank you & Best regards<br>
            </p>
          </body>
          </font>
        </html>
        """
        
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = perx_opt_email
        msg['Cc'] = cc_email
        
        msg['Subject'] = subject_success
        text = MIMEText(body_success, 'html')
        msg.attach(text)
        
        # Attach File in email
        files = [opt_file_name]
        for f in files:
            with open(f, "rb") as file:
                part = MIMEApplication(file.read(), Name=basename(f))
        # After the file is closed
        part['Content-Disposition'] = 'attachment; filename="%s"' % basename(f)
        msg.attach(part)
        
        # Sending the email
        try:
            smtp = smtplib.SMTP('smtprelay.siampiwat.com', 25)
            smtp.sendmail(sender_email, rcpt, msg.as_string())
            smtp.quit()
            print("Successfully sent email")
        except smtplib.SMTPException:
            print("Error: unable to send email")

# Run Program
### ระวังเรื่องชื่อ Column (profileid, account_id_ctm, email)

# วาง SQL ตรงนี้

In [117]:
sql_phrase = '''
WITH icon_txn AS
(
	SELECT
		uid
		, mobile_number 
		, userid_dl AS account_id_ctm
		, age 
		, profileid
		, trans_date
		, trans_year 
		, shopname
		, industryname
		, subindustryname
		, buildingname
		, region_new
		, outcome
		, point_transaction 
	FROM "SPWGSPS_VLC".viz_newsystem
	WHERE 1
	AND loyalty_program_id = 1
	AND coalesce(outcome,'x') <> 'expiry'
	AND lower(buildingname) LIKE any(ARRAY['%icon%', '%ics%'])
	AND trans_date >= current_date - INTERVAL '2 years' 
	AND classification_mem_group NOT LIKE '%VP%'
	AND unsubscribe_sms = 0
)
--SELECT DISTINCT subindustryname FROM icon_txn
, group_2 AS
(
	SELECT 
		*
		, 'Group 2' AS lead_group
	FROM
		(
			SELECT 
				*
				, ROW_NUMBER () OVER (ORDER BY num_visit DESC, last_visit_date ASC) AS rnk
			FROM
				(
					SELECT
						uid
						, profileid
						, mobile_number
						, account_id_ctm
						, count(DISTINCT trans_date) AS num_visit
						, max(trans_date) AS last_visit_date
						, EXTRACT(DAY FROM current_date - max(trans_date)) AS num_days
						, sum(point_transaction) AS spending
					FROM icon_txn
					WHERE 1
					AND subindustryname IN ('Books, Stationery, Art Supply', 'School & Academy', 'Play Land & Themed Attraction', 'Mom & Kids', 'Play Land & Themed Attraction')
					AND region_new IN ('Bangkok', 'Vicinity')
					GROUP BY 1,2,3,4
				)
			WHERE 1
			AND num_days < 366
			AND uid IS NOT NULL
		)
	WHERE rnk <= 20000
)
, group_1 AS
(
	SELECT 
		*
		, 'Group 1' AS lead_group
	FROM
		(
			SELECT 
				*
				, ROW_NUMBER () OVER (ORDER BY num_visit DESC, last_visit_date ASC) AS rnk
			FROM
				(
					SELECT
						ict.uid
						, ict.profileid
						, ict.mobile_number
						, ict.account_id_ctm
						, count(DISTINCT trans_date) AS num_visit
						, max(trans_date) AS last_visit_date
						, EXTRACT(DAY FROM current_date - max(trans_date)) AS num_days
						, sum(point_transaction) AS spending
					FROM icon_txn ict
					LEFT JOIN group_2 g2
					ON ict.uid = g2.uid
					WHERE 1
					and g2.uid IS NULL
					AND age < 45
					AND ict.uid IS NOT NULL
	--				AND uid NOT IN (SELECT uid FROM group_2)
	--				AND subindustryname IN ('Books, Stationery, Art Supply', 'School & Academy', 'Play Land & Themed Attraction', 'Mom & Kids', 'Play Land & Themed Attraction')
	--				AND region_new IN ('Bangkok', 'Vicinity')
					GROUP BY 1,2,3,4
				)
			WHERE 1
			AND num_days < 366
		)
		WHERE rnk <= 20000
)
--SELECT uid, count(*), count(DISTINCT uid)
SELECT 
	*
FROM
	(
		SELECT
			*
		FROM
			(
				SELECT 
					*
					, ROW_NUMBER () OVER (PARTITION BY uid ORDER BY lead_group DESC) AS selected_rnk
				FROM
					(
						SELECT * FROM group_1
						UNION ALL 
						SELECT * FROM group_2
					)
			)
		WHERE 1
		AND selected_rnk = 1
	)
WHERE 1
AND lead_group = 'Group 2'
ORDER BY rnk
'''

In [118]:
# ตั้งชื่อ subject
subject = lead_subject_provide()

# เลือก type lead
lead_selection, lead_text = choose_type_of_lead()
# เลือก เป็น lead with opt รึเปล่า
is_opt = opt_lead_check()

# ---------------------------
# input ว่าใครเป็นคนทำ --------
# In Case want to fix name : declare variable here and close lead provider function
lead_provide = 'Nutthapol W'
# lead_provide = lead_provider()
# clear_output()
# ----------------------------

# เลือกเจ้าของ lead
corp_selected, corp_directory = choose_corp()
# clear_output()

# ตั้งชื่อ
file_names_list, lead_name = naming_lead()
# clear_output()
file_path_temp = absolute_file_paths(file_path)
delete_temp_files()

# Query data
initial_df = query_postgres(sql_phrase)

# write excel และ upload
write_excel()
success_params, file_name_for_email = upload_file()

# ส่ง email แจ้งกรณี completed
send_email()

# ส่ง email กรณีเป็น opt lead
opt_file_name = write_opt_excel()
# send_email หา opt
send_email_opt()

Please input subject name เอาจากหัว Ticket:  [Lead Test]
Please select type of lead generate :
1 = sms only
2 = app push only
3 = social ads only
4 = sms and app push
 4
Do you to mail mobile number separately to perx operation team? 1 = Yes, 0 = No  :  0
Please select corporate :
1 = SPWG
2 = PRHG
3 = ICON
4 = SPS
 1


You selected SPWG
You selected sms and app push


Please naming your lead file :  lead_2


Your file name is : lead_2_sms.xlsx and lead_2_app_push.xlsx
Deleted /home/jovyan/48_lead_generation/temp_files/test2_sms.xlsx completed


/tmp/ipykernel_1633741/2477150719.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql_text , pg_conn)


lead_2_sms.xlsx uploaded successfully!
lead_2_app_push.xlsx uploaded successfully!
Successfully sent email
